# Logistic Regression for Predicting Cat Images

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage

## Pre-processing Data

In [ ]:
# defined a function to import training and testing datasets

def load_dataset():
    train_dataset = h5py.File('train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig

In [ ]:
# load in the data
X_train_orig, Y_train, X_test_orig, Y_test = load_dataset()

X_train_orig.shape

In [ ]:
idx = 60
plt.imshow(X_train_orig[idx])
Y_train[:,idx]

In [ ]:
# reshape inputs in a form suitable for our application

X_train_flatten = X_train_orig.reshape(X_train_orig.shape[0], -1).T
X_test_flatten = X_test_orig.reshape(X_test_orig.shape[0], -1).T

print(X_train_flatten.shape, X_test_flatten.shape)

In [ ]:
# for RGB images, standardize data by just dividing every colour value by 255

X_train = X_train_flatten / 255
X_test = X_test_flatten / 255

In [ ]:
X_train[:,2]

## Defining parts of the model

In [ ]:
# define some small data and parameters to test each part
X = np.array([[1,2], [3,4]])
Y = np.array([[1, 0]])
w = np.array([[1], [2]])
b = 2

X

In [ ]:
def sigmoid(z):
    s = 1 / (1 + np.exp(-z))
    return s

In [ ]:
sigmoid(X)

In [ ]:
# define forward and backward propagation

def propagate(w, b, X, Y):
    m = X.shape[1]
    
    # FORWARD PROPAGATION (FROM X TO COST)
    A = sigmoid(np.dot(w.T, X) + b)  # compute activation
    cost = (- 1 / m) * np.sum(Y * np.log(A) + (1 - Y) * (np.log(1 - A)))  # compute cost
    
    # BACKWARD PROPAGATION (TO FIND GRADIENTS)
    dZ = A - Y
    dw = (1 / m) * np.dot(X, dZ.T)
    db = (1 / m) * np.sum(dZ)

    cost = np.squeeze(cost) # removes unnecessary dimensions
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

In [ ]:
grads, cost = propagate(w, b, X, Y)
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print ("cost = " + str(cost))

In [ ]:
# define a function for gradient descent optimization

def optimize(X, Y, w, b, iters, learning_rate):
    costs = []
    
    for i in range(iters):
        # calculate costs and gradients for current iteration
        grads, cost = propagate(w, b, X, Y)
        dw = grads["dw"]
        db = grads["db"]
        
        # gradient descent update rule
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        # Record the costs every 100 iterations
        if i % 100 == 0:
            costs.append(cost)
            print ("Cost after iteration %i: %f" % (i, cost))
        
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [ ]:
params, grads, costs = optimize(X, Y, w, b, iters = 1000, learning_rate = 0.009)

print ("w = " + str(params["w"]))
print ("b = " + str(params["b"]))
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))

In [ ]:
# function for model prediction on an example

def predict(w, b, X):
    m = X.shape[1]
    Y_prediction = np.zeros((1, m)) # initialize with zeros
    w = w.reshape(X.shape[0], 1)
    
    # Compute vector "A" predicting the probabilities of a cat being present in the picture
    A = sigmoid(np.dot(w.T, X) + b)
    
    # Convert probabilities a[0,i] to actual predictions p[0,i]
    for i in range(A.shape[1]):
        if A[0, i] > 0.5:
            Y_prediction[0, i] = 1 
        else:
            Y_prediction[0, i] = 0
    
    return Y_prediction

In [ ]:
predict(w, b, X)

## Actual model

In [ ]:
# combine all the functions above into a model

def model(X_train, Y_train, X_test, Y_test, num_iterations=2000, learning_rate=0.5):
    
    # initialize parameters with zeros
    dim = X_train.shape[0] # no. of features

    w = np.zeros((dim, 1))
    b = 0

    # Gradient descent
    params, grads, costs = optimize(X_train, Y_train, w, b, num_iterations, learning_rate)
                           # will perform forward and backpropagation, gradient descent, costs etc.
    w = params["w"]
    b = params["b"]
    
    # Predict test/train set examples
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)

    # Print train/test Errors
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations} # contains all info about the model
    
    return d

In [ ]:
d = model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.005)

In [ ]:
# Predictions for the test set
index = 49
plt.imshow(X_test[:, index].reshape((64, 64, 3)))
print ("y = " + str(Y_test[0, index]))
print("predicted = " + str(d["Y_prediction_test"][0, index]))

In [ ]:
# Plot learning curve (with costs)
costs = np.squeeze(d['costs'])
plt.plot(costs)
plt.ylabel('cost')
plt.xlabel('iterations (hundreds)')
plt.title("Learning rate =" + str(d["learning_rate"]))
plt.show()

In [ ]:
# See how changing learning rates changes the accuracies

learning_rates = [0.01, 0.001, 0.0001]
models = {}
for i in learning_rates:
    print ("learning rate is: " + str(i))
    models[str(i)] = model(X_train, Y_train, X_test, Y_test, num_iterations = 1500, learning_rate = i)
    print ('\n\n')

for i in learning_rates:
    plt.plot(np.squeeze(models[str(i)]["costs"]), label= str(models[str(i)]["learning_rate"]))

plt.ylabel('cost')
plt.xlabel('iterations')
plt.legend()

In [ ]:
d_test = model(X_train, Y_train, X_test, Y_test, num_iterations = 5000, learning_rate = 0.001)